In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

dico_annee = {"2023_2024" : ["Auxerre", "Angers", "Saint-Étienne", "Rodez", "Paris FC"],
              "2022_2023" : ["Le Havre", "Metz", "Bordeaux", "Bastia", "Caen"],
              "2021_2022" : ["Toulouse", "AC Ajaccio", "Auxerre", "Paris FC", "Sochaux"],
              "2020_2021" : ["Troyes", "Clermont Foot", "Toulouse", "Grenoble Foot", "Paris FC"]}

In [2]:
annee = list(dico_annee.keys())[0]

In [3]:
event_import = pd.read_json(f"../../Data_file/Heatmap SB/{annee}.json")

In [16]:
event = event_import[~event_import.location.isna()]

event.sort_values(by = ["match_id", "index"], inplace = True)

event.index = range(len(event))

centre = event[(event.pass_cross == 1) & (event.pass_type != "Corner")]

dico_event_test = {"Nb_but" : {}, "Event avant but" : {}}
for n_event in range (3, 7) :
    liste_event = []
    count_goal = 0
    for i in centre.index :
        team_i = centre.team
        event_i = event.loc[i + 1 : i + n_event]
        if sum(((event_i.shot_outcome == "Goal") | (event_i.type == "Own Goal Against")) & (event_i.team == centre.loc[i, "team"])) > 0 :
            loc_goal = event_i[((event_i.shot_outcome == "Goal") | (event_i.type == "Own Goal Against")) & (event_i.team == centre.loc[i, "team"])].index.values
            liste_event.append(event.loc[i + 1 : loc_goal[0], "type"].tolist())
            count_goal += 1
        dico_event_test["Nb_but"][n_event] = count_goal
        dico_event_test["Event avant but"][n_event] = liste_event

# df = pd.concat([centre.match_id, centre.team, centre.goal, pd.DataFrame(centre.location.tolist(), index = centre.index), pd.DataFrame(centre.pass_end_location.tolist(), index = centre.index)], axis = 1, ignore_index = True)
# df.columns = ["match_id", "Équipe", "goal", "x", "y", "x_end", "y_end"]

# df["Top 5"] = df["Équipe"].isin(dico_annee[annee])

In [24]:
df_event_test = pd.DataFrame(index = range(3, 7))

In [31]:
df_event_test = pd.DataFrame(index = range(3, 7))
for key in dico_event_test["Event avant but"].keys() :
    for i in dico_event_test["Event avant but"][key] :
        for j in i :
            if j not in df_event_test.columns :
                df_event_test = pd.concat([df_event_test, pd.Series(0, index = df_event_test.index, name = j)], axis = 1)
            df_event_test.loc[key, j] += 1

In [32]:
df_event_test

,Ball Receipt*,Duel,Shot,Carry,Pressure,Error,Block,Ball Recovery,Goal Keeper,Clearance,Foul Committed,Pass,Miscontrol,Dribbled Past,Dribble,Foul Won
3,204,21,209,10,34,1,2,3,2,1,1,0,0,0,0,0
4,222,23,227,16,41,1,7,10,2,7,2,2,0,0,0,0
5,261,25,262,29,51,5,10,25,10,17,2,12,1,0,0,0
6,290,30,288,37,66,6,14,35,17,21,5,20,2,1,1,3


In [37]:
pd.Series(dico_event_test["Nb_but"], name = "Nombre but")

3    209
4    227
5    257
6    278
Name: Nombre but, dtype: int64

In [42]:
pd.concat([pd.Series(dico_event_test["Nb_but"], name = "Nombre but"), df_event_test], axis = 1).T

,3,4,5,6
Nombre but,209,227,257,278
Ball Receipt*,204,222,261,290
Duel,21,23,25,30
Shot,209,227,262,288
Carry,10,16,29,37
Pressure,34,41,51,66
Error,1,1,5,6
Block,2,7,10,14
Ball Recovery,3,10,25,35
Goal Keeper,2,2,10,17


In [28]:
df = pd.concat([centre.match_id, centre.team, centre.goal, pd.DataFrame(centre.location.tolist(), index = centre.index), pd.DataFrame(centre.pass_end_location.tolist(), index = centre.index)], axis = 1, ignore_index = True)
df

,0,1,2,3,4,5,6
14,3896583,AC Ajaccio,NaN,116.8,67.8,111.7,46.6
105,3896583,AC Ajaccio,NaN,86.5,7.2,111.4,44.0
163,3896583,Rodez,NaN,99.1,71.8,107.0,48.3
322,3896583,AC Ajaccio,NaN,90.5,76.9,105.1,47.5
369,3896583,AC Ajaccio,NaN,106.8,21.1,113.4,37.7
...,...,...,...,...,...,...,...
1340065,3937846,Rodez,NaN,107.1,10.0,106.6,48.4
1340087,3937846,Saint-Étienne,NaN,118.4,54.7,108.7,36.1
1340130,3937846,Saint-Étienne,NaN,116.3,65.8,105.5,0.1
1340183,3937846,Rodez,NaN,98.2,74.2,106.5,44.9


In [1]:


df.columns = ["match_id", "Équipe", "goal", "x", "y", "x_end", "y_end"]

df["Top 5"] = df["Équipe"].isin(dico_annee[annee])

NameError: name 'dico_annee' is not defined